In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark=SparkSession.builder.appName('EcomDataPipeline').getOrCreate()

In [0]:
silver_sellers = spark.read.format("delta").load("/mnt/delta/tables/silver/sellers")
silver_buyers = spark.read.format("delta").load("/mnt/delta/tables/silver/buyers")
silver_users = spark.read.format("delta").load("/mnt/delta/tables/silver/users")
silver_countries = spark.read.format("delta").load("/mnt/delta/tables/silver/countries")

In [0]:
# merge all tables into one big table using 'country' as the common coloumn
comprehensive_table=silver_users \
                    .join(silver_buyers,['country'],'outer') \
                    .join(silver_sellers,['country'],'outer') \
                    .join(silver_countries,['country'],'outer')

In [0]:
# change the naming of the columns for ease of readability
comprehensive_table=comprehensive_table.select(
    silver_users["country"].alias("Country"),
    # From silver_users
    silver_users["productsSold"].alias("Users_productsSold"),
    silver_users["productsWished"].alias("Users_productsWished"),
    silver_users["account_age_years"].alias("Users_account_age_years"),
    silver_users["account_age_group"].alias("Users_account_age_group"),
    silver_users["hasanyapp"].alias("Users_hasanyapp"),
    silver_users["socialnbfollowers"].alias("Users_socialnbfollowers"),
    silver_users["flag_long_title"].alias("Users_flag_long_title"),
    # Continue with other silver_users columns as needed...
    
    # From silver_countries
    silver_countries["sellers"].alias("Countries_Sellers"),
    silver_countries["topsellers"].alias("Countries_TopSellers"),
    silver_countries["femalesellers"].alias("Countries_FemaleSellers"),
    silver_countries["malesellers"].alias("Countries_MaleSellers"),
    silver_countries["topfemalesellers"].alias("Countries_TopFemaleSellers"),
    silver_countries["topmalesellers"].alias("Countries_TopMaleSellers"),
    # Continue with other silver_countries columns as needed...
    
    # From silver_buyers
    silver_buyers["buyers"].alias("Buyers_Total"),
    silver_buyers["topbuyers"].alias("Buyers_Top"),
    silver_buyers["femalebuyers"].alias("Buyers_Female"),
    silver_buyers["malebuyers"].alias("Buyers_Male"),
    silver_buyers["topfemalebuyers"].alias("Buyers_TopFemale"),
    silver_buyers["topmalebuyers"].alias("Buyers_TopMale"),
    # Continue with other silver_buyers columns as needed...
    
    # From silver_sellers
    silver_sellers["nbsellers"].alias("Sellers_Total"),
    silver_sellers["sex"].alias("Sellers_Sex"),
    silver_sellers["meanproductssold"].alias("Sellers_MeanProductsSold"),
    silver_sellers["meanproductslisted"].alias("Sellers_MeanProductsListed"),
    # Continue with other silver_sellers columns as needed...
)



In [0]:
comprehensive_table.write.mode('overwrite').format('delta').save('/mnt/delta/tables/gold/comprehensive_obt')

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2304521126126519>, line 1
----> 1 comprehensive_table.write.mode('overwrite').format('delta').save('/mnt/delta/tables/golg/comprehensive_obt')

NameError: name 'comprehensive_table' is not defined

In [0]:
# copies files to processed/ and removes from to_process/
source_path='dbfs:/mnt/ecomdata/users-raw2/to_process/'
destination_path='dbfs:/mnt/ecomdata/users-raw2/processed/'
files=dbutils.fs.ls(source_path)
for file in files:
    dbutils.fs.cp(file.path,destination_path + file.name)
for file in files:
    if file.isFile():
        dbutils.fs.rm(file.path)


In [0]:
%sql
create or replace table comprehensive_obt_table
using delta
as
select * from
delta.`/mnt/delta/tables/gold/comprehensive_obt`;

num_affected_rows,num_inserted_rows
